In [1]:
from torchvision.models import vgg
from torch_lib.Model_heading_bin import *
from torch_lib.ClassAverages import *
from torchvision import transforms
import os, glob, cv2
from library.ron_utils import *
import matplotlib.pyplot as plt
import numpy as np

def plot_regressed_3d_bbox(img, cam_to_img, box2d, dimensions, alpha, theta_ray, detectionid):

    # the math! returns X, the corners used for constraint
    location, X = calc_location(dimensions, cam_to_img, box2d, alpha, theta_ray)

    orient = alpha + theta_ray

    #plot_2d_box(img, box2d, detectionid)
    plot_3d_box(img, cam_to_img, orient, dimensions, location) # 3d boxes

    return location, orient

#REG PNG TO VIDEO
import cv2, os, glob
def make_video(folder, img_paths, reg_folder, fps=15):
    video_name = os.path.join(folder, f'{reg_folder}.avi')
    image0 = os.path.join(folder, img_paths[0].replace('images', f'{reg_folder}'))
    frame = cv2.imread(image0)
    height, width, layers = frame.shape
    video = cv2.VideoWriter(video_name, 0, fps=fps, frameSize=(width,height))
    for i in range(len(img_paths)):
        path = os.path.join(folder, img_paths[i].replace('images', reg_folder))
        video.write(cv2.imread(path))
    cv2.destroyAllWindows()
    video.release()    

def make_compare_video(folder, img_paths, reg_folder1, reg_folder2, fps=15):
    video_name = os.path.join(folder, f'C_{reg_folder1}_{reg_folder2}.avi')
    image0 = os.path.join(folder, img_paths[0])
    frame = cv2.imread(image0)
    height, width, layers = frame.shape
    video = cv2.VideoWriter(video_name, 0, fps=fps, frameSize=(width*2, height))
    for i in range(len(img_paths)):
        path1 = os.path.join(folder, img_paths[i].replace('images', reg_folder1))
        path2 = os.path.join(folder, img_paths[i].replace('images', reg_folder2))
        img1 = cv2.imread(path1)
        img2 = cv2.imread(path2)
        compare = np.concatenate((img1, img2), axis=1) #horizontal
        video.write(compare)
    cv2.destroyAllWindows()
    video.release()

def round_list(list_, float_point=2):
    foo = list()
    for i in range(len(list_)):
        new_val = round(float(list_[i]), float_point)
        foo.append(new_val)
    return foo

# Eval all folders

In [10]:
import json
# init setting
weights_list = ['DA_B4_N1_50', 'BL_B4_N1_50']
ELAN_averages = ClassAverages(average_file='all_ELAN_class_averages.txt')
cam_to_img = np.array([
        [1.418667e+03, 0.000e+00, 6.4e+02, 0],
        [0.000e+00, 1.418867e+03, 3.6e+02, 0],
        [0.000e+00, 000e+00, 1.0e+00, 0] ])
class_dict = {8:'truck', 9:'car', 10:'motor', 11:'bike'}
device = torch.device('cuda') # 選gpu的index

for weights_name in weights_list:
    weights_path = f'weights/{weights_name}.pkl'

    checkpoint = torch.load(weights_path, map_location=device) #if training on 2 GPU, mapping on the same device
    normalize_type = checkpoint['normal']
    bin_num = checkpoint['bin']
    angle_per_class = 2*np.pi/float(bin_num)

    my_vgg = vgg.vgg19_bn(weights='DEFAULT').to(device)
    model = Model(features=my_vgg.features, bins=bin_num).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])
    # for img processing
    if normalize_type == 0:
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
    if normalize_type == 1:
        normalize = transforms.Normalize(mean=[0.596, 0.612, 0.587], std=[0.256, 0.254, 0.257])
    process = transforms.Compose([transforms.ToTensor(), 
                                  transforms.Resize([224,224], transforms.InterpolationMode.BICUBIC), 
                                  normalize])


    valid_folder = 'IVAvalid_for_NCTU'
    for time in ['day', 'night']:
        folder = f'{valid_folder}/{time}'
        for sub_f in os.listdir(folder):
            sub_folder = os.path.join(folder, sub_f)
            with open(f'{sub_folder}/TEST_images.json') as f:
                img_paths = json.load(f)
            label_json = f'{sub_folder}/TEST_objects.json'
            with open(label_json) as f:
                label_dicts = json.load(f)

            os.makedirs(os.path.join(sub_folder, weights_name), exist_ok=True)
            result = list()
            model.eval()
            with torch.no_grad():
                for i in range(len(img_paths)):

                    path = os.path.join(sub_folder, img_paths[i])
                    img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
                    label_dict = label_dicts[i]

                    save_path = path.replace('images', weights_name)
                    result_json = os.path.join(sub_folder, f'REG_result_{weights_name}.json')
                    result_dict = dict()
                    result_dict['alpha'] = list()
                    result_dict['theta_ray'] = list()
                    result_dict['dimension'] = list()
                    result_dict['location'] = list()
                    for j in range(len(label_dict['labels'])):
                        difficulty = label_dict['difficulties'][j] # 1 is hard to distinguish
                        if difficulty == 0:
                            color = (255, 0, 0)
                            line_width = 2
                        else:
                            color = (0, 0, 255)
                            line_width = 1
                        class_ = class_dict[label_dict['labels'][j]] # 5:行人,6:機車騎士,7:腳踏車騎士,8:大車,9:小車,10:機車,11:腳踏車
                        object_id = label_dict['id'][j] # for tracking
                        box2d = label_dict['boxes'][j]
                        box2d = [[box2d[0], box2d[1]], [box2d[2],box2d[3]]]
                        left_top = box2d[0]
                        right_btm = box2d[1]

                        crop = img[left_top[1]:right_btm[1]+1, left_top[0]:right_btm[0]+1] 
                        crop = process(crop) 
                        crop = torch.stack([crop]).to(device)

                        [RESIDUALs, BIN_CONFs, delta_DIMs] = model(crop)
                        bin_argmax = torch.max(BIN_CONFs, dim=1)[1]
                        orient_residual = RESIDUALs[torch.arange(len(RESIDUALs)), bin_argmax] 
                        Alphas = angle_per_class*bin_argmax + orient_residual #mapping bin_class and residual to get alpha
                        reg_alpha = float(Alphas[0].data)
                        reg_alpha = round(angle_correction(reg_alpha), 2)
                        reg_dim = delta_DIMs.cpu().data.numpy()[0, :]
                        reg_dim += ELAN_averages.get_item(class_)
                        
                        theta_ray = round(calc_theta_ray(img.shape[1], box2d, cam_to_img), 2)
                        loc, ry = plot_regressed_3d_bbox(img, cam_to_img, box2d, reg_dim, reg_alpha, theta_ray, object_id)
                        reg_dim = round_list(reg_dim, 2)
                        loc = round_list(loc, 2)
                        result_dict['alpha'].append(reg_alpha)
                        result_dict['theta_ray'].append(theta_ray)
                        result_dict['dimension'].append(reg_dim)
                        result_dict['location'].append(loc)
                        
                    result.append(result_dict)
                    #plt.imsave(save_path, img)
                    #plt.show()
                with open(result_json, 'w') as f:
                    json.dump(result, f)
                print(f'DONE {sub_folder}')
            #make_video(sub_folder, img_paths, weights_name, fps=15)
print('ALL FINISHED')

DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-091818-824
DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-092149-828
DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-092421-831
DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230226-143139-2863
DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230226-143946-2887
DONE IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-202628-892
DONE IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-202954-900
DONE IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-203217-905
DONE IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-203246-908
DONE IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-203322-911
DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-091818-824
DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-092149-828
DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-092421-831
DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230226-143139-2863
DONE IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230226-143946-2887
DONE IVAvalid_for_NCTU/night\c32b95371b6b

### make COMPARE video only (given folder)

In [10]:
import cv2, os, glob
valid_folder = 'IVAvalid_for_NCTU'
reg_folders = ['R_DA_B4_N1_50', 'R_all_BL_B4_N1_50']
for time in ['day', 'night']:
    folder = f'{valid_folder}/{time}'
    for sub_f in os.listdir(folder):
        sub_folder = os.path.join(folder, sub_f)
        with open(f'{sub_folder}/TEST_images.json') as f:
            img_paths = json.load(f)
        make_compare_video(sub_folder, img_paths, reg_folders[0], reg_folders[1])
        print(f'Done {sub_folder}')
print('ALL FINISHED')

Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-091818-824
Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-092149-828
Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230209-092421-831
Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230226-143139-2863
Done IVAvalid_for_NCTU/day\c32b95371b6b-SOS20230226-143946-2887
Done IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-202628-892
Done IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-202954-900
Done IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-203217-905
Done IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-203246-908
Done IVAvalid_for_NCTU/night\c32b95371b6b-SOS20230210-203322-911
ALL FINISHED


## Reproduce by REG_result.json

In [16]:
import json, torch, os, cv2
from library.Plotting import *
import numpy as np
import matplotlib.pyplot as plt
# init setting
weights_list = ['DA_B4_N1_50', 'all_BL_B4_N1_50']
cam_to_img = np.array([
        [1.418667e+03, 0.000e+00, 6.4e+02, 0],
        [0.000e+00, 1.418867e+03, 3.6e+02, 0],
        [0.000e+00, 000e+00, 1.0e+00, 0] ])
class_dict = {8:'truck', 9:'car', 10:'motor', 11:'bike'}
device = torch.device('cuda') # 選gpu的index

valid_folder = 'IVAvalid_for_NCTU'
# editable
prefix = 'Box2d'
for weights_name in weights_list:
    for time in ['day', 'night']:
        folder = f'{valid_folder}/{time}'
        for sub_f in os.listdir(folder):
            sub_folder = os.path.join(folder, sub_f)
            with open(f'{sub_folder}/TEST_images.json') as f:
                img_paths = json.load(f)
            label_json = f'{sub_folder}/TEST_objects.json'
            with open(label_json) as f:
                label_dicts = json.load(f)
            
            result_json = f'{sub_folder}/REG_result_{weights_name}.json'
            with open(result_json) as f:
                result_dicts = json.load(f)
            
            re_folder = os.path.join(sub_folder, f'{prefix}_{weights_name}')
            os.makedirs(re_folder, exist_ok=True)
            for i in range(len(img_paths)):
                path = os.path.join(sub_folder, img_paths[i])
                save_path = path.replace('images', f'{prefix}_{weights_name}')
                
                img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
                img_box2d = np.copy(img)
                label_dict = label_dicts[i]
                result_values = result_dicts[i]
                for j in range(len(label_dict['labels'])):
                    orient = result_values['alpha'][j] + result_values['theta_ray'][j]
                    dim = result_values['dimension'][j]
                    loc = result_values['location'][j]
                    plot_3d_box(img, cam_to_img, orient, dim, loc) # 3d boxes
                    box2d = label_dict['boxes'][j]
                    cv2.rectangle(img_box2d, (box2d[0],box2d[1]), (box2d[2],box2d[3]), (0,0,255), 2)
                    #plot_2d_box(img_box2d, box2d)
                    concat = np.concatenate((img_box2d, img), axis=1)
                    #plt.imshow(concat)
                    #plt.show()
                plt.imsave(save_path, concat)
            make_video(sub_folder, img_paths, f'{prefix}_{weights_name}', fps=15)
            print(f'Done {weights_name} | {sub_folder.split("/")[-1]}')
print('All finished')

Done DA_B4_N1_50 | day\c32b95371b6b-SOS20230209-091818-824
Done DA_B4_N1_50 | day\c32b95371b6b-SOS20230209-092149-828
Done DA_B4_N1_50 | day\c32b95371b6b-SOS20230209-092421-831
Done DA_B4_N1_50 | day\c32b95371b6b-SOS20230226-143139-2863
Done DA_B4_N1_50 | day\c32b95371b6b-SOS20230226-143946-2887
Done DA_B4_N1_50 | night\c32b95371b6b-SOS20230210-202628-892
Done DA_B4_N1_50 | night\c32b95371b6b-SOS20230210-202954-900
Done DA_B4_N1_50 | night\c32b95371b6b-SOS20230210-203217-905
Done DA_B4_N1_50 | night\c32b95371b6b-SOS20230210-203246-908
Done DA_B4_N1_50 | night\c32b95371b6b-SOS20230210-203322-911
Done all_BL_B4_N1_50 | day\c32b95371b6b-SOS20230209-091818-824
Done all_BL_B4_N1_50 | day\c32b95371b6b-SOS20230209-092149-828
Done all_BL_B4_N1_50 | day\c32b95371b6b-SOS20230209-092421-831
Done all_BL_B4_N1_50 | day\c32b95371b6b-SOS20230226-143139-2863
Done all_BL_B4_N1_50 | day\c32b95371b6b-SOS20230226-143946-2887
Done all_BL_B4_N1_50 | night\c32b95371b6b-SOS20230210-202628-892
Done all_BL_B4_N

## BOX_3D with BEV

In [8]:
import glob, torch, os
import matplotlib.pyplot as plt
from library.Plotting import *
import json
from library.ron_utils import calc_depth_with_alpha_theta
# init setting
weights_list = ['DA_B4_N1_50', 'all_BL_B4_N1_50']
cam_to_img = np.array([
        [1.418667e+03, 0.000e+00, 6.4e+02, 0],
        [0.000e+00, 1.418867e+03, 3.6e+02, 0],
        [0.000e+00, 000e+00, 1.0e+00, 0] ])
class_dict = {8:'truck', 9:'car', 10:'motor', 11:'bike'}
device = torch.device('cuda') # 選gpu的index

valid_folder = 'IVAvalid_for_NCTU'
BEV_size=(720, 360)
SCALE = 6
dis_interval = 10 # every circle means 10m
prefix = 'calc_depth'
for weights_name in weights_list:
    for time in ['day', 'night']:
        folder = f'{valid_folder}/{time}'
        for sub_f in os.listdir(folder):
            sub_folder = os.path.join(folder, sub_f)
            with open(f'{sub_folder}/TEST_images.json') as f:
                img_paths = json.load(f)
            label_json = f'{sub_folder}/TEST_objects.json'
            with open(label_json) as f:
                label_dicts = json.load(f)
            
            result_json = f'{sub_folder}/REG_result_{weights_name}.json'
            with open(result_json) as f:
                result_dicts = json.load(f)
            
            re_folder = os.path.join(sub_folder, f'{prefix}_{weights_name}')
            os.makedirs(re_folder, exist_ok=True)
            for i in range(len(img_paths)):
                path = os.path.join(sub_folder, img_paths[i])
                save_path = path.replace('images', f'{prefix}_{weights_name}')
                
                img = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
                img_W = img.shape[1]
                label_dict = label_dicts[i]
                result_values = result_dicts[i]
                image_bev = my_init_bev(scale=SCALE, bev_size=BEV_size, dis_interval=10, thickness=4)
                for j in range(len(label_dict['labels'])):
                    alpha = result_values['alpha'][j]
                    orient = result_values['alpha'][j] + result_values['theta_ray'][j]
                    dim = result_values['dimension'][j]
                    box2d = label_dict['boxes'][j]
                    loc = result_values['location'][j]
                    calc_depth = calc_depth_with_alpha_theta(img_W, box2d, cam_to_img, dim[1], dim[2], alpha, trun=0)
                    #print(loc[2], calc_depth)
                    plot_3d_box(img, cam_to_img, orient, dim, loc) # 3d boxes
                    x, z = loc[0], calc_depth#loc[2]
                    W, L = dim[1], dim[2]
                    bev_rect = my_get_bev_rect(image_bev, x, z, L, W, orient, scale=SCALE, bev_size=BEV_size)
                    draw_bev_rect(image_bev, bev_rect)
                visual_result = np.concatenate((img, image_bev), axis=1)
#                plt.imshow(visual_result)
#                plt.show()
#                break
                plt.imsave(save_path, visual_result)
#            break
            make_video(sub_folder, img_paths, f'{prefix}_{weights_name}', fps=10)
            print(f'Done {weights_name} | {sub_folder.split("/")[-1]}')
#        break
print('All finished')

Done DA_B4_N1_50 | day\c32b95371b6b-SOS20230209-091818-824
Done DA_B4_N1_50 | day\c32b95371b6b-SOS20230209-092149-828
Done DA_B4_N1_50 | day\c32b95371b6b-SOS20230209-092421-831
Done DA_B4_N1_50 | day\c32b95371b6b-SOS20230226-143139-2863
Done DA_B4_N1_50 | day\c32b95371b6b-SOS20230226-143946-2887
Done DA_B4_N1_50 | night\c32b95371b6b-SOS20230210-202628-892
Done DA_B4_N1_50 | night\c32b95371b6b-SOS20230210-202954-900
Done DA_B4_N1_50 | night\c32b95371b6b-SOS20230210-203217-905
Done DA_B4_N1_50 | night\c32b95371b6b-SOS20230210-203246-908
Done DA_B4_N1_50 | night\c32b95371b6b-SOS20230210-203322-911
Done all_BL_B4_N1_50 | day\c32b95371b6b-SOS20230209-091818-824
Done all_BL_B4_N1_50 | day\c32b95371b6b-SOS20230209-092149-828
Done all_BL_B4_N1_50 | day\c32b95371b6b-SOS20230209-092421-831
Done all_BL_B4_N1_50 | day\c32b95371b6b-SOS20230226-143139-2863
Done all_BL_B4_N1_50 | day\c32b95371b6b-SOS20230226-143946-2887
Done all_BL_B4_N1_50 | night\c32b95371b6b-SOS20230210-202628-892
Done all_BL_B4_N

### MAKE vid with BEV for tommy

In [1]:
import glob, os, math, cv2
import numpy as np
import matplotlib.pyplot as plt

# BEV FUNCs
def init_bev(scale=10, bev_size=(720, 360), dis_interval=5, thickness=2):
    x_offset = bev_size[1] // 2 #圓心位置 (車子位置)
    image_bev = np.zeros((bev_size[0], bev_size[1], 3), np.uint8)
    for i in range(1, 20): #黑底白圈
        cv2.circle(image_bev, (x_offset, bev_size[0]), scale*i*dis_interval, (255, 255, 255), thickness)
    return image_bev

def rotate_point(x, y, center_x, center_y, theta):
    x = x - center_x
    y = y - center_y
    nx = int(center_x + x * math.cos(theta) - y * math.sin(theta))
    ny = int(center_y + x * math.sin(theta) + y * math.cos(theta))
    return nx, ny

def draw_bev_rect(image, rect, thickness=2):
    center_x = rect[0]
    center_y = rect[1]
    w = rect[2]
    h = rect[3]
    theta = rect[4]
    x1 = center_x - 0.5 * w 
    x2 = center_x + 0.5 * w 
    y1 = center_y - 0.5 * h 
    y2 = center_y + 0.5 * h 

    point_list = []
    point_list.append(rotate_point(x1, y1, center_x, center_y, theta))
    point_list.append(rotate_point(x1, y2, center_x, center_y, theta))
    point_list.append(rotate_point(x2, y2, center_x, center_y, theta))
    point_list.append(rotate_point(x2, y1, center_x, center_y, theta))

    red = (255, 0, 0)
    blue = (0, 0, 255)
    cv2.line(image, point_list[0], point_list[1], blue, thickness)
    cv2.line(image, point_list[1], point_list[2], blue, thickness)
    cv2.line(image, point_list[2], point_list[3], red, thickness*2) #紅色表示車頭朝向
    cv2.line(image, point_list[3], point_list[0], blue, thickness)

def get_bev_rect(location, dimension, orient, scale, bev_size=(720, 360)):
    x3d, _, z3d = location
    _, w3d, l3d = dimension
    x_offset = bev_size[1] // 2
    bev_rect = [0, 0, 0, 0, 0]
    bev_rect[0] = x3d * scale + x_offset
    bev_rect[1] = bev_size[0] - z3d * scale
    bev_rect[2] = l3d * scale
    bev_rect[3] = w3d * scale
    bev_rect[4] = orient
    return bev_rect

# IMAGES to VIDEO
def make_video(img_folder, vid_name, fps=15):
    video_name = os.path.join(img_folder, f'{vid_name}.avi')
    images = sorted(glob.glob(f'{img_folder}/*.png'))
    height, width, _ = cv2.imread(images[0]).shape
    video = cv2.VideoWriter(video_name, 0, fps=fps, frameSize=(width,height))
    for i in range(len(images)):
        video.write(cv2.imread(images[i]))
    cv2.destroyAllWindows()
    video.release()   

cam_to_img = [[1.418667e+03, 0.000e+00, 6.4e+02, 0],
            [0.000e+00, 1.418867e+03, 3.6e+02, 0],
            [0.000e+00, 000e+00, 1.0e+00, 0] ]

data_root = 'Elan_3d_box'
images = sorted(glob.glob(f'{data_root}/image_2/*.png'))
labels = sorted(glob.glob(f'{data_root}/label_2/*.txt'))
BEV_image_folder = f'{data_root}/image_bev'
os.makedirs(BEV_image_folder, exist_ok=True)
H, _, _= cv2.imread(images[0]).shape
BEV_SIZE=(H, H//2+100) # 2:1方便看, 第一個值是為了concat img, 第二個可以是any value
SCALE = 6 # 越大會讓可視的最遠距離越小)
INTERVAL = 10 # 一個白圈表示的距離
FPS = 5 # video fps
for i in range(len(images)):
    save_path = os.path.join(BEV_image_folder, f'{i:06d}.png')
    lines = open(labels[i]).readlines()
    img = cv2.cvtColor(cv2.imread(images[i]), cv2.COLOR_BGR2RGB)
    img_bev = init_bev(scale=SCALE, bev_size=BEV_SIZE, dis_interval=INTERVAL, thickness=4)
    for line in lines:
        line = line.split()
        if line[0].lower() == 'dontcare':
            continue
        for j in range(1,15):
            line[j] = float(line[j])

        box2d = [[int(line[4]), int(line[5])], [int(line[6]), int(line[7])]]
        dim = [line[8], line[9], line[10]]
        loc = [line[11], line[12], line[13]]
        global_orient = float(line[14])
        #plot_3d_box(img, cam_to_img, global_orient, dim, loc) # 3d boxes
        bev_rect = get_bev_rect(loc, dim, global_orient, scale=SCALE, bev_size=BEV_SIZE)
        draw_bev_rect(img_bev, bev_rect)
        cv2.rectangle(img, box2d[0], box2d[1], (255,0,0), 2)
    concat_result = np.concatenate((img, img_bev), axis=1)
    plt.imsave(save_path, concat_result)
    #plt.imshow(concat_result)
    #print(save_path)
make_video(img_folder=BEV_image_folder, vid_name='BEV', fps=5)

KeyboardInterrupt: 

In [4]:
# from MonoDet
import math
def rotate_point(x, y, center_x, center_y, theta):
    x = x - center_x
    y = y - center_y
    nx = int(center_x + x * math.cos(theta) - y * math.sin(theta))
    ny = int(center_y + x * math.sin(theta) + y * math.cos(theta))
    return nx, ny

def draw_bev_rect(image, rect, thickness=2):
    center_x = rect[0]
    center_y = rect[1]
    w = rect[2]
    h = rect[3]
    theta = rect[4]
    x1 = center_x - 0.5 * w 
    x2 = center_x + 0.5 * w 
    y1 = center_y - 0.5 * h 
    y2 = center_y + 0.5 * h 

    point_list = []
    point_list.append(rotate_point(x1, y1, center_x, center_y, theta))
    point_list.append(rotate_point(x1, y2, center_x, center_y, theta))
    point_list.append(rotate_point(x2, y2, center_x, center_y, theta))
    point_list.append(rotate_point(x2, y1, center_x, center_y, theta))

    red = (255, 0, 0)
    blue = (0, 0, 255)
    cv2.line(image, point_list[0], point_list[1], blue, thickness)
    cv2.line(image, point_list[1], point_list[2], blue, thickness)
    cv2.line(image, point_list[2], point_list[3], red, thickness*2)
    cv2.line(image, point_list[3], point_list[0], blue, thickness)

def init_bev(scale=10, bev_size=(720, 360), dis_interval=5, thickness=2):
    x_offset = bev_size[1] // 2
    image_bev = np.zeros((bev_size[0], bev_size[1], 3), np.uint8)
    for i in range(1, 20):
        cv2.circle(image_bev, (x_offset, bev_size[0]), scale * i * dis_interval, (255, 255, 255), thickness)
    return image_bev

def get_bev_rect(image_bev, x3d, z3d, l3d, w3d, ry3d, scale=10, bev_size=(720, 360)):
    #bev_size = image_bev.shape
    x_offset = bev_size[1] // 2
    bev_rect = [0, 0, 0, 0, 0]
    bev_rect[0] = x3d * scale + x_offset
    bev_rect[1] = bev_size[0] - z3d * scale
    bev_rect[2] = l3d * scale
    bev_rect[3] = w3d * scale
    bev_rect[4] = ry3d
    return bev_rect

## Elan_3d_box_230808 with reg_labels

In [5]:
import matplotlib.pyplot as plt
import cv2
# init setting
weights_list = ['DA_B4_N1_50', 'BL_B4_N1_50']
cam_to_img = np.array([
        [1.418667e+03, 0.000e+00, 6.4e+02, 0],
        [0.000e+00, 1.418867e+03, 3.6e+02, 0],
        [0.000e+00, 000e+00, 1.0e+00, 0] ])

valid_folder = 'Elan_3d_box_230808'
BEV_size=(720, 360)
SCALE = 6
FPS = 5
dis_interval = 10 # every circle means 10m

for weights_name in weights_list:
    img_paths = glob.glob(f'{valid_folder}/image_2/*.png')
    label_paths = glob.glob(f'{valid_folder}/{weights_name}/label_2/*.txt')
    plt_image_folder = f'{valid_folder}/{weights_name}/image_2'
    os.makedirs(plt_image_folder, exist_ok=True)
    
    for i in range(len(img_paths)):
        image_bev = my_init_bev(scale=SCALE, bev_size=BEV_size, dis_interval=10, thickness=4)
        save_path = img_paths[i].replace(valid_folder, f'{valid_folder}/{weights_name}')
        img = cv2.imread(img_paths[i])
        lines = open(label_paths[i]).readlines()
        for line in lines:
            elements = line.split()
            for j in range(1, len(elements)):
                elements[j] = float(elements[j])
            orient = elements[14]
            dim = [elements[8], elements[9], elements[10]]
            loc = [elements[11], elements[12], elements[13]]
            plot_3d_box(img, cam_to_img, orient, dim, loc) # 3d boxes
            x, z = loc[0], loc[2] #calc_depth
            W, L = dim[1], dim[2]
            bev_rect = my_get_bev_rect(image_bev, x, z, L, W, orient, scale=SCALE, bev_size=BEV_size)
            draw_bev_rect(image_bev, bev_rect)
        
        visual_result = np.concatenate((img, image_bev), axis=1)
        plt.imsave(save_path, visual_result)
        #plt.imshow(visual_result)
        #plt.show()
    print(f'DONE {weights_name}')
    #make_video(sub_folder, img_paths, weights_name, fps=15)
    
    video_name = os.path.join(valid_folder, f'{weights_name}.avi')
    image0 = img_paths[0].replace(valid_folder, f'{valid_folder}/{weights_name}')
    frame = cv2.imread(image0)
    height, width, layers = frame.shape
    video = cv2.VideoWriter(video_name, 0, fps=FPS, frameSize=(width,height))
    for i in range(len(img_paths)):
        path = img_paths[i].replace(valid_folder, f'{valid_folder}/{weights_name}')
        video.write(cv2.imread(path))
    cv2.destroyAllWindows()
    video.release()
print('ALL FINISHED')

IndexError: list index out of range